In [ ]:
!pip install tensorboard
!pip install gensim
!pip install pyLDAvis
!pip install wordcloud matplotlib
!pip install emoji
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
!gzip -d cc.en.300.bin.gz
!pip install fasttext

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 9.2 MB/s eta 0:00:00
--2025-02-25 12:05:12--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.35.7.38, 13.35.7.82, 13.35.7.50, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.35.7.38|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4503593528 (4.2G) [application/octet-stream]
Saving to: ‘cc.en.300.bin.gz’

cc.en.300.bin.gz    100%[===================>]   4.19G   139MB/s    in 34s     

2025-02-25 12:05:47 (126 MB/s) - ‘cc.en.300.bin.gz’ saved [4503593528/4503593528]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 2.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.6-py3-none-any.

In [ ]:
import nltk
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
nltk.download('stopwords')
nltk.download('punkt')
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
import re
import nltk
from nltk.corpus import stopwords
from collections import defaultdict, Counter
import pandas as pd
import emoji
from gensim.models import Word2Vec

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
!wget -P data/full_dataset/ https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_1.csv
!wget -P data/full_dataset/ https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_2.csv
!wget -P data/full_dataset/ https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_3.csv

--2025-02-25 12:07:41--  https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_1.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.125.207, 142.250.157.207, 142.251.8.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.125.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14174600 (14M) [application/octet-stream]
Saving to: ‘data/full_dataset/goemotions_1.csv’

goemotions_1.csv    100%[===================>]  13.52M  8.32MB/s    in 1.6s    

2025-02-25 12:07:43 (8.32 MB/s) - ‘data/full_dataset/goemotions_1.csv’ saved [14174600/14174600]

--2025-02-25 12:07:43--  https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_2.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.125.207, 142.250.157.207, 142.251.8.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.125.207|:443... connected.
HTTP request sent, awaiting r

In [ ]:
goemotions_1 = pd.read_csv('data/full_dataset/goemotions_1.csv')
goemotions_2 = pd.read_csv('data/full_dataset/goemotions_2.csv')
goemotions_3 = pd.read_csv('data/full_dataset/goemotions_3.csv')

combined_df = pd.concat([goemotions_1, goemotions_2, goemotions_3], ignore_index=True)
emotion_columns = combined_df.columns[9:]

combined_df['created_utc'] = pd.to_datetime(combined_df['created_utc'], unit='s', errors='coerce')
combined_df.to_csv('data/full_dataset/goemotions_combined.csv', index=False)
combined_df = combined_df.query("example_very_unclear != True")

#work white emoji...

In [ ]:

# Dictionary to store emojis with their positions and texts
emoji_dict = {}

# Function to extract emojis and build the dictionary
def build_emoji_dict(df):
    for idx, row in df.iterrows():
        text = row['text']
        for char in text:
            if emoji.is_emoji(char):
                if char not in emoji_dict:
                    emoji_dict[char] = []
                emoji_dict[char].append((idx, text))
build_emoji_dict(combined_df)


#Process

In [ ]:
def preprocess_text(text):
    text = re.sub(r"[<>]", "", text)
    text = f"[START] {text} [END]"

    text = re.sub(r'\[(\w+)\]', r'<\1>', text)
    text = emoji.demojize(text)
    text = re.sub(r"[^a-zA-Z0-9<> ]", "", text)

    text = text.lower()
    return text

combined_df['processed_text'] = combined_df['text'].apply(preprocess_text)

print(combined_df[['text', 'processed_text']].head())

                                                text  \
0                                    That game hurt.   
2     You do right, if you don't care then fuck 'em!   
3                                 Man I love reddit.   
4  [NAME] was nowhere near them, he was by the Fa...   
5  Right? Considering it’s such an important docu...   

                                      processed_text  
0                       <start> that game hurt <end>  
2  <start> you do right if you dont care then fuc...  
3                    <start> man i love reddit <end>  
4  <start> <name> was nowhere near them he was by...  
5  <start> right considering its such an importan...  


In [ ]:
def create_mappings(df):
    word_to_index = {}
    index_to_word = {}
    index_counter = 1

    for text in df['processed_text']:
        words = text.split()

        for word in words:
            if word not in word_to_index:
                word_to_index[word] = index_counter
                index_to_word[index_counter] = word
                index_counter += 1

    return word_to_index, index_to_word


word_to_index, index_to_word = create_mappings(combined_df)

print("Word to Index Mapping:", dict(list(word_to_index.items())[:10]))
print("Index to Word Mapping:", dict(list(index_to_word.items())[:10]))

Word to Index Mapping: {'<start>': 1, 'that': 2, 'game': 3, 'hurt': 4, '<end>': 5, 'you': 6, 'do': 7, 'right': 8, 'if': 9, 'dont': 10}
Index to Word Mapping: {1: '<start>', 2: 'that', 3: 'game', 4: 'hurt', 5: '<end>', 6: 'you', 7: 'do', 8: 'right', 9: 'if', 10: 'dont'}


In [ ]:
len(word_to_index),len(index_to_word)

(33701, 33701)

In [ ]:
def text_to_index_sequence(text, word_to_index):
    words = text.split()
    index_sequence = [word_to_index[word] for word in words if word in word_to_index]
    return index_sequence
combined_df['text_as_indexes'] = combined_df['processed_text'].apply(lambda x: text_to_index_sequence(x, word_to_index))
print(combined_df[['text', 'processed_text', 'text_as_indexes']].head())

                                                text  \
0                                    That game hurt.   
2     You do right, if you don't care then fuck 'em!   
3                                 Man I love reddit.   
4  [NAME] was nowhere near them, he was by the Fa...   
5  Right? Considering it’s such an important docu...   

                                      processed_text  \
0                       <start> that game hurt <end>   
2  <start> you do right if you dont care then fuc...   
3                    <start> man i love reddit <end>   
4  <start> <name> was nowhere near them he was by...   
5  <start> right considering its such an importan...   

                                     text_as_indexes  
0                                    [1, 2, 3, 4, 5]  
2          [1, 6, 7, 8, 9, 6, 10, 11, 12, 13, 14, 5]  
3                             [1, 15, 16, 17, 18, 5]  
4     [1, 19, 20, 21, 22, 23, 24, 20, 25, 26, 27, 5]  
5  [1, 8, 28, 29, 30, 31, 32, 33, 16, 34, 35, 26,..

#word2Vec

In [ ]:
def prepare_data_for_word2vec(df):
    processed_texts = df['processed_text'].apply(lambda x: x.split())
    return processed_texts

sentences = prepare_data_for_word2vec(combined_df)
model = Word2Vec(sentences, vector_size=300, window=5, min_count=1, workers=4)
word_vectors = {word: model.wv[word] for word in model.wv.index_to_key}
print("Word vector for 'start':", word_vectors['start'])

#FestText

In [ ]:
import fasttext
fasttext_model = fasttext.load_model('/content/cc.en.300.bin')

def prepare_data_for_fasttext(df):
    processed_texts = df['processed_text'].apply(lambda x: x.split())
    return processed_texts

sentences = prepare_data_for_fasttext(combined_df)

word_vectors = {word: fasttext_model.get_word_vector(word) for sentence in sentences for word in sentence}

print("Word vector for 'start':", word_vectors.get('start', 'Word not found'))


Word vector for 'start': [ 7.91064836e-03 -9.43343155e-03  2.28888560e-02  1.15760624e-01
 -5.44784889e-02  3.52381244e-02  1.51470797e-02  9.68480669e-03
  3.72007266e-02 -8.26112553e-03 -3.63847502e-02  7.02371597e-02
 -1.53566375e-02  1.11012265e-01  1.33964136e-01  6.41972050e-02
  6.07942492e-02 -1.30296145e-02  1.28229149e-04  9.35186446e-02
 -6.10206239e-02 -2.02715285e-02 -1.50931003e-02 -9.10209268e-02
  1.57276485e-02 -1.85630284e-02 -6.97859824e-02  6.44239336e-02
 -4.34500724e-02  3.08100097e-02  2.29679141e-03 -5.58396354e-02
 -6.58368096e-02 -6.34431988e-02  3.13772000e-02  2.18865033e-02
  3.18698958e-02  1.05855837e-01 -2.08669528e-03 -9.21282396e-02
  3.49404141e-02 -9.22180153e-03  8.44760537e-02  6.59895763e-02
 -3.71814519e-02  7.32254684e-02  3.71225476e-02 -6.29411414e-02
 -3.49417143e-02 -3.72454687e-03  1.67517923e-02  1.74221359e-02
  8.67096111e-02 -5.21334670e-02 -9.95707512e-03 -1.62409935e-02
  3.27702239e-02  3.58919427e-03 -8.23421478e-02 -1.64883211e-03


#work white the labell

In [ ]:
from sklearn.model_selection import train_test_split

def split_data_for_emotion(combined_df, emotion_columns, emotion, test_size=0.2, random_state=42):
    # Separate the entries for the specific emotion
    emotion_entries = combined_df[combined_df[emotion] == 1].copy()

    # Set other emotion columns to 0
    for col in emotion_columns:
        if col != emotion:
            emotion_entries[col] = 0

    # Combine non-specific emotion entries
    all_other_emotions = pd.concat([combined_df[combined_df[e] == 1] for e in emotion_columns if e != emotion])

    # Handle potential list-type columns
    for col in all_other_emotions.columns:
        if all_other_emotions[col].apply(lambda x: isinstance(x, list)).any():
            all_other_emotions[col] = all_other_emotions[col].apply(lambda x: str(x) if isinstance(x, list) else x)

    # Remove duplicates
    all_other_emotions = all_other_emotions.drop_duplicates()

    # Split data into train and test
    train_emotion, test_emotion = train_test_split(emotion_entries, test_size=test_size, random_state=random_state)
    train_other, test_other = train_test_split(all_other_emotions, test_size=test_size, random_state=random_state)

    # Filter to keep only processed_text and current emotion
    train_emotion_filtered = train_emotion[['processed_text', emotion]]
    test_emotion_filtered = test_emotion[['processed_text', emotion]]
    train_other_filtered = train_other[['processed_text', emotion]]
    test_other_filtered = test_other[['processed_text', emotion]]

    # Combine emotion and non-emotion entries
    train_combined = pd.concat([train_emotion_filtered, train_other_filtered], ignore_index=True)
    test_combined = pd.concat([test_emotion_filtered, test_other_filtered], ignore_index=True)

    print(f"{emotion} - Train: {len(train_combined)}, Test: {len(test_combined)}")

    return train_combined, test_combined

# Apply for all emotions
emotion_data = {}
for emotion in emotion_columns:
    if emotion in combined_df.columns:
        train, test = split_data_for_emotion(combined_df, emotion_columns, emotion)
        emotion_data[emotion] = {'train': train, 'test': test}
    else:
        print(f"Emotion {emotion} not found in the dataset.")

print("Data splitting complete for all emotions!")


admiration - Train: 171530, Test: 42884
amusement - Train: 168743, Test: 42186
anger - Train: 168556, Test: 42140
annoyance - Train: 170471, Test: 42619
approval - Train: 171340, Test: 42835
caring - Train: 168231, Test: 42059
confusion - Train: 168187, Test: 42048
curiosity - Train: 169296, Test: 42325
desire - Train: 167586, Test: 41898
disappointment - Train: 169261, Test: 42316
disapproval - Train: 169241, Test: 42311
disgust - Train: 168160, Test: 42041
embarrassment - Train: 167084, Test: 41773
excitement - Train: 168338, Test: 42085
fear - Train: 167385, Test: 41848
gratitude - Train: 169891, Test: 42473
grief - Train: 166508, Test: 41628
joy - Train: 169174, Test: 42294
love - Train: 168837, Test: 42211
nervousness - Train: 167062, Test: 41766
optimism - Train: 169608, Test: 42402
pride - Train: 166740, Test: 41686
realization - Train: 169508, Test: 42377
relief - Train: 166651, Test: 41664
remorse - Train: 167063, Test: 41766
sadness - Train: 168595, Test: 42150
surprise - Tra

In [ ]:
emotion_data.keys()

dict_keys(['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral'])

#Preparing the data for input into the model

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.model_selection import StratifiedKFold

class EmotionDataset(Dataset):
    def __init__(self, texts, labels, word_vectors, max_len=100):
        self.texts = texts
        self.labels = labels
        self.word_vectors = word_vectors
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        emotion_label = self.labels[idx]

        indexed_text = [self.word_vectors.get(word, np.zeros(300)) for word in text.split()]

        indexed_text = indexed_text[:self.max_len] if len(indexed_text) > self.max_len else indexed_text + [np.zeros(300)] * (self.max_len - len(indexed_text))

        indexed_text = torch.tensor(np.array(indexed_text), dtype=torch.float)
        emotion_label = torch.tensor(emotion_label, dtype=torch.float)

        return indexed_text, emotion_label

# Dictionary to store dataloaders for each emotion
emotion_dataloaders = {}

k = 5
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

for emotion in emotion_columns:
    train_texts = emotion_data[emotion]['train']['processed_text'].tolist()
    train_labels = emotion_data[emotion]['train'][emotion].tolist()

    test_texts = emotion_data[emotion]['test']['processed_text'].tolist()
    test_labels = emotion_data[emotion]['test'][emotion].tolist()

    test_dataset = EmotionDataset(test_texts, test_labels, word_vectors)
    test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    emotion_dataloaders[emotion] = {'test': test_dataloader}

    X = np.array(train_texts)
    y = np.array(train_labels)

    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
        print(f"{emotion} - Training on Fold {fold+1}...")

        X_train_fold, X_val_fold = X[train_idx], X[val_idx]
        y_train_fold, y_val_fold = y[train_idx], y[val_idx]

        train_dataset_fold = EmotionDataset(X_train_fold, y_train_fold, word_vectors)
        val_dataset_fold = EmotionDataset(X_val_fold, y_val_fold, word_vectors)

        train_dataloader_fold = DataLoader(train_dataset_fold, batch_size=32, shuffle=True)
        val_dataloader_fold = DataLoader(val_dataset_fold, batch_size=32, shuffle=False)

        if 'folds' not in emotion_dataloaders[emotion]:
            emotion_dataloaders[emotion]['folds'] = []

        emotion_dataloaders[emotion]['folds'].append({'train': train_dataloader_fold, 'val': val_dataloader_fold})

print("Dataloaders ready for all emotions!")


admiration - Training on Fold 1...
admiration - Training on Fold 2...
admiration - Training on Fold 3...
admiration - Training on Fold 4...
admiration - Training on Fold 5...
amusement - Training on Fold 1...
amusement - Training on Fold 2...
amusement - Training on Fold 3...
amusement - Training on Fold 4...
amusement - Training on Fold 5...
anger - Training on Fold 1...
anger - Training on Fold 2...
anger - Training on Fold 3...
anger - Training on Fold 4...
anger - Training on Fold 5...
annoyance - Training on Fold 1...
annoyance - Training on Fold 2...
annoyance - Training on Fold 3...
annoyance - Training on Fold 4...
annoyance - Training on Fold 5...
approval - Training on Fold 1...
approval - Training on Fold 2...
approval - Training on Fold 3...
approval - Training on Fold 4...
approval - Training on Fold 5...
caring - Training on Fold 1...
caring - Training on Fold 2...
caring - Training on Fold 3...
caring - Training on Fold 4...
caring - Training on Fold 5...
confusion - Tra

In [ ]:
import torch
import torch.nn as nn

class EmotionLSTMWithAttention(nn.Module):
    def __init__(self, input_dim=300, hidden_dim=128, output_dim=1, n_layers=2, dropout=0.5):
        super(EmotionLSTMWithAttention, self).__init__()
        self.input_dropout = nn.Dropout(dropout)
        # Bidirectional LSTM Layer
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=n_layers, batch_first=True, dropout=dropout, bidirectional=True)

        # Attention Mechanism
        self.attention = nn.Linear(hidden_dim * 2, 1)  # Attention mechanism to calculate attention scores
        self.attn_dropout = nn.Dropout(dropout)
        # LayerNorm after LSTM
        self.ln = nn.LayerNorm(hidden_dim * 2)  # Since bidirectional, we multiply hidden_dim by 2

        # Fully connected layer
        self.fc = nn.Linear(hidden_dim * 2, output_dim)

        # Dropout after the LSTM and fully connected layer
        self.dropout = nn.Dropout(dropout)

    def attention_mechanism(self, lstm_output):
        """
        Applies attention mechanism over the LSTM outputs.
        """
        attention_scores = self.attention(lstm_output)  # Shape: (batch_size, seq_len, 1)
        attention_weights = torch.softmax(attention_scores, dim=1)  # Shape: (batch_size, seq_len, 1)
        weighted_sum = torch.sum(attention_weights * lstm_output, dim=1)  # Shape: (batch_size, hidden_dim*2)
        weighted_sum = self.attn_dropout(weighted_sum)
        return weighted_sum

    def forward(self, x):
        # x is the input tensor of shape (batch_size, seq_len, input_dim)
        x = self.input_dropout(x)
        # LSTM forward pass
        lstm_out, (hidden, cell) = self.lstm(x)

        # Apply attention mechanism
        attention_out = self.attention_mechanism(lstm_out)

        # Layer normalization
        attention_out = self.ln(attention_out)

        # Pass through fully connected layer and dropout
        out = self.fc(attention_out)
        out = self.dropout(out)

        return out

# Instantiate the model with the desired parameters
emotion_models = {}

for emotion in emotion_columns:
    model = EmotionLSTMWithAttention(input_dim=300, hidden_dim=128, output_dim=1, n_layers=2, dropout=0.5)
    emotion_models[emotion] = model

In [ ]:
import time
import torch
import torch.optim as optim
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader, WeightedRandomSampler
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Focal Loss Implementation
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2, reduction='mean', pos_weight=None):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction
        self.pos_weight = pos_weight

    def forward(self, inputs, targets):
        BCE_loss = nn.BCEWithLogitsLoss(pos_weight=self.pos_weight, reduction='none')(inputs, targets)
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1 - pt) ** self.gamma * BCE_loss

        if self.reduction == 'mean':
            return F_loss.mean()
        elif self.reduction == 'sum':
            return F_loss.sum()
        else:
            return F_loss

# Training function with additional metrics, early stopping, and LR scheduler
def simple_train(model, train_dataloader_fold, val_dataloader_fold, criterion, optimizer, epochs=5):
    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []

    val_precisions = []
    val_recalls = []
    val_f1s = []
    val_aucs = []

    start_time = time.time()

    for epoch in range(epochs):
        model.train()
        epoch_train_loss = 0
        epoch_train_acc = 0

        for inputs, labels in train_dataloader_fold:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(inputs)
            labels = labels.float()
            loss = criterion(outputs.squeeze(), labels)
            epoch_train_loss += loss.item()

            preds = torch.round(torch.sigmoid(outputs)).detach().cpu().numpy()
            batch_acc = f1_score(labels.detach().cpu().numpy(), preds, average="macro")
            epoch_train_acc += batch_acc

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

        avg_train_loss = epoch_train_loss / len(train_dataloader_fold)
        avg_train_acc = epoch_train_acc / len(train_dataloader_fold)
        train_losses.append(avg_train_loss)
        train_accuracies.append(avg_train_acc)

        model.eval()
        epoch_val_loss = 0
        epoch_val_acc = 0
        all_preds = []
        all_labels = []
        with torch.no_grad():
            for inputs, labels in val_dataloader_fold:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                labels = labels.float()
                loss = criterion(outputs.squeeze(), labels)
                epoch_val_loss += loss.item()

                preds = torch.round(torch.sigmoid(outputs)).detach().cpu().numpy()
                batch_acc = f1_score(labels.detach().cpu().numpy(), preds, average="macro")
                epoch_val_acc += batch_acc

                all_preds.extend(preds)
                all_labels.extend(labels.detach().cpu().numpy())

        avg_val_loss = epoch_val_loss / len(val_dataloader_fold)
        avg_val_acc = epoch_val_acc / len(val_dataloader_fold)
        val_losses.append(avg_val_loss)
        val_accuracies.append(avg_val_acc)

        epoch_val_f1 = f1_score(all_labels, all_preds, average='macro')
        epoch_val_precision = precision_score(all_labels, all_preds, average='macro')
        epoch_val_recall = recall_score(all_labels, all_preds, average='macro')
        try:
            epoch_val_auc = roc_auc_score(all_labels, all_preds)
        except Exception:
            epoch_val_auc = 0.0

        val_f1s.append(epoch_val_f1)
        val_precisions.append(epoch_val_precision)
        val_recalls.append(epoch_val_recall)
        val_aucs.append(epoch_val_auc)

        epoch_time = time.time() - start_time
        print(f"Epoch {epoch+1}/{epochs} - Time: {epoch_time:.2f}s")

        if avg_val_loss < best_loss:
            best_loss = avg_val_loss
            torch.save(model.state_dict(), f'best_model_{label}.pt')
            print("  Saved the best model!")

        if epochs_without_improvement >= patience:
            print("  Early stopping triggered. Stopping training.")
            break

    return train_losses, val_losses, train_accuracies, val_accuracies, val_precisions, val_recalls, val_f1s, val_aucs

# Training loop for all emotion models
for label, model in emotion_models.items():
    print(f"Starting training for emotion: {label}")

    train_labels = train_combined[label].tolist()
    val_labels = val_combined[label].tolist()

    label_counts = np.bincount(train_labels)
    weights = 1.0 / label_counts
    sample_weights = weights[train_labels]
    sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)

    pos_count = np.sum(train_labels)
    neg_count = len(train_labels) - pos_count
    pos_weight = torch.tensor([neg_count / pos_count]).to(device)

    criterion = FocalLoss(alpha=0.20, gamma=2.5, pos_weight=pos_weight)

    train_dataset = EmotionDataset(train_combined['processed_text'].tolist(), train_labels, word_vectors)
    val_dataset = EmotionDataset(val_combined['processed_text'].tolist(), val_labels, word_vectors)

    train_dataloader = DataLoader(train_dataset, batch_size=32, sampler=sampler)
    val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)

    optimizer = optim.AdamW(model.parameters(), lr=0.0001, weight_decay=0.01)

    results = simple_train(model, train_dataloader, val_dataloader, criterion, optimizer, epochs=20)


#Test for each model

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, roc_auc_score, confusion_matrix, precision_score, recall_score
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter(log_dir='./runs/evaluation_results')

# Evaluate test metrics function
def evaluate_test_metrics(model, label_name, test_dataloader, criterion):
    model.eval()
    all_labels = []
    all_outputs = []

    with torch.no_grad():
        for inputs, labels in test_dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)

            all_labels.extend(labels.cpu().numpy())
            all_outputs.extend(torch.sigmoid(outputs).cpu().numpy())

    all_labels = np.array(all_labels)
    all_outputs = np.array(all_outputs)

    # Calculating metrics
    f1 = f1_score(all_labels, (all_outputs > 0.4).astype(int))
    roc_auc = roc_auc_score(all_labels, all_outputs)
    accuracy = np.mean((all_labels == (all_outputs > 0.4).astype(int)))

    # Confusion matrix
    cm = confusion_matrix(all_labels, (all_outputs > 0.4).astype(int))

    # Precision and recall
    precision = precision_score(all_labels, (all_outputs > 0.7).astype(int))
    recall = recall_score(all_labels, (all_outputs > 0.3).astype(int))

    # Specificity
    tn, fp, fn, tp = cm.ravel()
    specificity = tn / (tn + fp) if (tn + fp) != 0 else 0

    # Printing results
    print(f"Label: {label_name}")
    print(f"Test F1 Score: {f1:.4f}")
    print(f"Test ROC AUC: {roc_auc:.4f}")
    print(f"Test Accuracy: {accuracy:.4f}")
    print(f"Test Precision: {precision:.4f}")
    print(f"Test Recall: {recall:.4f}")
    print(f"Test Specificity: {specificity:.4f}\n")

    # Logging results in TensorBoard
    writer.add_scalar(f'{label_name}/Test F1 Score', f1)
    writer.add_scalar(f'{label_name}/Test ROC AUC', roc_auc)
    writer.add_scalar(f'{label_name}/Test Accuracy', accuracy)
    writer.add_scalar(f'{label_name}/Test Precision', precision)
    writer.add_scalar(f'{label_name}/Test Recall', recall)
    writer.add_scalar(f'{label_name}/Test Specificity', specificity)

    # Plot confusion matrix
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["Negative", "Positive"], yticklabels=["Negative", "Positive"])
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.title(f"Confusion Matrix for {label_name}")
    plt.show()

# Dictionary of models per label
label_models = {
    'joy': joy_model,
    'sadness': sadness_model,
    'anger': anger_model,
    'fear': fear_model
}

# Evaluating each model
for label, model in label_models.items():
    evaluate_test_metrics(model, label, test_dataloader, criterion)

# Closing the TensorBoard writer
writer.close()


#the score

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Function to get the probabilities for a new text
def get_emotion_probabilities(text, emotion_models):
    probabilities = {}

    for label, model in emotion_models.items():
        model.eval()
        # Assuming 'tokens' is the representation of the text as tokens (already preprocessed)
        tokens = preprocess_text(text)  # Assuming you have such a function
        tokens_tensor = torch.tensor(tokens).to(device).unsqueeze(0)

        with torch.no_grad():
            output = model(tokens_tensor)
            prob = torch.sigmoid(output).cpu().numpy()[0]  # Get the probability

        probabilities[label] = prob

    # Create a DataFrame with the probabilities
    df = pd.DataFrame(probabilities, index=[0])

    # Normalize the probabilities using MinMaxScaler
    scaler = MinMaxScaler()
    normalized_probabilities = scaler.fit_transform(df)

    # Add unnormalized and normalized columns to the DataFrame
    df_normalized = pd.DataFrame(normalized_probabilities, columns=df.columns)
    df_normalized["normalized"] = df_normalized.values.tolist()

    df["unnormalized"] = df.values.tolist()

    return df, df_normalized

# Example usage
text = "I'm really happy today"
df, df_normalized = get_emotion_probabilities(text, emotion_models)

# Displaying unnormalized probabilities
print("Probabilities (Unnormalized):")
print(df)

# Displaying normalized probabilities
print("Probabilities (Normalized):")
print(df_normalized)


#white llm

In [ ]:
ef get_emotion_probabilities(text, emotion_models):
    emotion_probs = []
    for model in emotion_models:
        inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True).to(device)
        emotion_prob = model(inputs['input_ids'], inputs['attention_mask']).squeeze().item()
        emotion_probs.append(emotion_prob)
    return torch.tensor(emotion_probs).unsqueeze(0).to(device)  # [1, num_emotions]

# Instantiate models
llm_model_name = "bert-base-uncased"  # Example LLM (you can choose a different one)
emotion_models = [EmotionModel() for _ in range(5)]  # Replace with real emotion models

# Create the Emotion-AUG LLM model
emotion_augmented_llm = EmotionAugmentedLLM(llm_model_name, emotion_models).to(device)

# Example text and emotion probabilities (for demo)
texts = ["I am so happy today!", "I feel so sad..."]
emotion_probs = [
    [0.9, 0.05, 0.1, 0.2, 0.05],  # Example emotion probabilities for the first text
    [0.1, 0.9, 0.2, 0.5, 0.05]   # Example emotion probabilities for the second text
]

# Prepare dataset and dataloader
dataset = EmotionTextDataset(texts, emotion_probs, tokenizer=AutoTokenizer.from_pretrained(llm_model_name))
dataloader = DataLoader(dataset, batch_size=2, shuffle=False)

# Training loop
optimizer = optim.Adam(emotion_augmented_llm.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()

for epoch in range(10):
    emotion_augmented_llm.train()
    running_loss = 0.0
    for text, emotion_labels in dataloader:
        optimizer.zero_grad()

        # Get emotion probabilities
        emotion_features = get_emotion_probabilities(text, emotion_models)

        # Forward pass
        outputs = emotion_augmented_llm(text, emotion_features)

        # Calculate loss and backpropagate
        loss = criterion(outputs, emotion_labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/10], Loss: {running_loss/len(dataloader)}")

# Save the model
torch.save(emotion_augmented_llm.state_dict(), "emotion_augmented_llm.pth")